In [26]:
# !pip install numpy
# !pip install pandas
# !pip install plotly
# !pip install nbformat --upgrade
# !pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.3/315.3 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.2 MB/s eta 0:00:0031m17.7 MB/s eta 0:00:01
  Created wheel for frozendict: filename=frozendict-2.4.2-cp311-cp311-macosx_10_9_universal2.whl size=15509 sha256=cb17ea598acd5de3e5340e1f191cf9db8679382dc1c769c0c4

In [32]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import yfinance as yf
import os

pio.renderers.default = 'browser'

In [39]:
def fetch_stock_data(tickers, start_date=None, end_date=None, interval='1d'):
    """
    Fetches historical stock data for given tickers using yfinance.
    
    Parameters:
    - tickers (list of str): List of stock ticker symbols.
    - start_date (str): Start date for the data in format 'YYYY-MM-DD' (optional).
    - end_date (str): End date for the data in format 'YYYY-MM-DD' (optional).
    - interval (str): Data interval. Valid intervals: '1d', '1wk', '1mo', etc.

    Returns:
    - dict: A dictionary with tickers as keys and DataFrames as values.
    """
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval=interval)
        stock_data[ticker] = data
    
    return stock_data

# Example usage
tickers = ['AAPL', 'GOOG', 'MSFT', 'TSLA', 'NVDA', 'BTC-USD'] 
data = fetch_stock_data(tickers, '2020-01-01', '2024-05-05', '1d')

In [40]:
data['NVDA'].head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02 00:00:00-05:00,59.452379,59.741238,58.946879,59.741238,23753600,0.0,0.0
2020-01-03 00:00:00-05:00,58.543478,59.223288,58.294462,58.785023,20538400,0.0,0.0
2020-01-06 00:00:00-05:00,57.851218,59.083842,57.589751,59.031548,26263600,0.0,0.0
2020-01-07 00:00:00-05:00,59.315428,60.204414,58.864711,59.746223,31485600,0.0,0.0
2020-01-08 00:00:00-05:00,59.703892,60.271646,59.302977,59.858284,27710800,0.0,0.0


# Normalized Growth Visualizer

Description:

    This function visualizes the normalized growth of multiple stocks over a specified date range using Plotly, an interactive graphing library. It creates a line chart where each line represents the normalized growth trajectory of a different stock. This allows for easy comparison of stock performance over time.

Output:

    An interactive line chart will be displayed. The chart is rendered in a web browser for an enhanced viewing experience, providing tools for zooming, panning, and toggling data series visibility.



In [49]:


def plotly_normalized_growth(dataframes, start_date, end_date):
    fig = go.Figure()

    # Convert user input dates to timezone-naive datetime, assuming dataframes contain timezone-aware datetimes
    start_date_naive = pd.to_datetime(start_date).tz_localize(None)
    end_date_naive = pd.to_datetime(end_date).tz_localize(None)

    for stock, df in dataframes.items():
        # Reset the index if 'Date' is not a column
        if 'Date' not in df.columns:
            df = df.reset_index()

        # Ensure the 'Date' column is of datetime type and is timezone-naive for consistency
        df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
        
        # Filter the dataframe for the given date range
        mask = (df['Date'] >= start_date_naive) & (df['Date'] <= end_date_naive)
        filtered_df = df.loc[mask]
        
        # Plot the normalized growth
        if not filtered_df.empty:
            fig.add_trace(go.Scatter(x=filtered_df['Date'], y=filtered_df['Normalized Growth'],
                                     mode='lines', name=stock))
    
    # Update plot layout
    fig.update_layout(
        title='Normalized Growth of Stocks Over Time',
        xaxis_title='Date',
        yaxis_title='Normalized Growth',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    
    # Show the figure
    fig.show()

# Example usage:
# Assuming 'data' is the dictionary of DataFrames with 'Date' possibly in the index
plotly_normalized_growth(data, "2020-01-02", "2020-06-01")



KeyError: 'Normalized Growth'